In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-11-11 07:00:00')


In [3]:
from src.helpers.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.
2023-11-11 08:34:18,560 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (8.73s) from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   


In [6]:
from src.helpers.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.
Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x7f19887b2480>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.790775035785763,
                               feature_fraction=0.5412877229956432,
                               min_child_samples=34, num_leaves=182))])
[LightGBM] [Warning] feature_fraction is set=0.5412877229956432, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5412877229956432
[LightGBM] [Warning] bagging_fraction is set=0.790775035785763, subsample=1.0 will be ignored. Current value: bagging_fraction=0.790775035785763


In [7]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2023-11-11 07:00:00
1,2,0.0,2023-11-11 07:00:00
2,3,0.0,2023-11-11 07:00:00
3,4,2.0,2023-11-11 07:00:00
4,5,0.0,2023-11-11 07:00:00
...,...,...,...
260,261,8.0,2023-11-11 07:00:00
261,262,48.0,2023-11-11 07:00:00
262,263,61.0,2023-11-11 07:00:00
263,264,19.0,2023-11-11 07:00:00


In [8]:
from src.helpers.feature_store_api import get_feature_store
import src.helpers.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/178327/fs/178246/fg/210251


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/178327/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f19e25bd6d0>, None)